In [13]:
import sqlite3


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip


In [9]:
sqliteConnection = sqlite3.connect('data.db')
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
  

In [10]:
cursor = sqliteConnection.cursor()

In [11]:
cursor.execute(sql_query)

In [12]:
print(cursor.fetchall())

[('balances',), ('client_products',), ('client',), ('inv_campaign_eval',)]


In [14]:
import pandas as pd

In [15]:
balances_query = """SELECT * FROM balances"""
balances_df = pd.read_sql_query(balances_query, sqliteConnection)

In [16]:
balances_df.head()

,date,balance,client_id,currency
0,2023-06-06 16:21:15.291346,2680.467773,249789938,CZK
1,2023-05-23 16:21:15.291346,2654.055419,249789938,CZK
2,2023-05-09 16:21:15.291346,2620.835872,249789938,CZK
3,2023-04-25 16:21:15.291346,2720.315925,249789938,CZK
4,2023-04-11 16:21:15.291346,2687.601982,249789938,CZK
